# PURPOSE OF THIS NOTEBOOK 

This notebook takes the data needed for the individual 22 models and trains all the models to work on

In [2]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
from transformers import BertTokenizer, BertModel, BertConfig
from pathlib import Path
from torch import cuda
import torch

from sklearn import metrics 
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
# Setting up for GPU 

device = 'cuda' if cuda.is_available() else 'cpu'

## TODO: 
I need to create a function that imports each of the 

In [ ]:
# import the 